In [2]:
pip install langchain langchain-community langchain-huggingface faiss-cpu smolagents markdownify duckduckgo-search python-docx unstructured requests pandas scikit-learn joblib gradio transformers

   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
    ----------------

In [13]:
import pandas as pd
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer
import joblib # Để tải mô hình ML
import numpy as np # Để xử lý output xAI


In [14]:
# --- Đường dẫn đến các file của bạn ---
PROCESSED_DATA_PATH = r'D:\Project\FraudGraphML\ML Model\Dataset\processed_data.csv'
TRANSACTION_DATA_PATH = r'D:\Project\FraudGraphML\ML Model\Dataset\transaction_dataset.csv'
TRAINED_MODEL_PATH = r'D:\Project\FraudGraphML\ML Model\Model\trained_model.pkl' # Mô hình phát hiện bất thường của bạn

In [ ]:
# --- Tải và chuẩn bị dữ liệu chính ---
print("Đang tải dữ liệu và chuẩn bị cho Knowledge Base...")

df_processed = pd.read_csv(PROCESSED_DATA_PATH)
df_transaction = pd.read_csv(TRANSACTION_DATA_PATH)

# Đảm bảo cột 'Index' là số nguyên để join
df_processed['Index'] = df_processed['Index'].astype(int)
df_transaction['Index'] = df_transaction['Index'].astype(int)

# Join hai dataframe để có cột 'Address' trong df_processed
# Sử dụng suffix để tránh xung đột tên cột nếu có
df_data = pd.merge(df_processed, df_transaction[['Index', 'Address']], on='Index', how='left', suffixes=('', '_trans'))

# Xử lý trường hợp cột 'ERC20 uniq sent addr.1' có thể là trùng lặp hoặc lỗi đánh máy
# Nếu nó là bản sao của 'ERC20 uniq sent addr', có thể loại bỏ nó hoặc đổi tên để tránh nhầm lẫn.
# Giả sử chúng ta sẽ giữ lại nó nếu nó có giá trị khác biệt, hoặc loại bỏ nếu nó hoàn toàn trùng lặp.
# Để đơn giản, tôi sẽ giả định nó là một trường độc lập cho mục đích này.

Đang tải dữ liệu và chuẩn bị cho Knowledge Base...


In [16]:
df_data.head()

,Index,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,...,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type_label,ERC20_most_rec_token_type_label,Address
0,1,0,844.26,1093.71,704785.63,721,89,0,40,118,...,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,63,299,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8
1,1,0,844.26,1093.71,704785.63,721,89,0,40,118,...,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,63,299,0x4c13f6966dc24c92489344f0fd6f0e61f3489b84
2,1,0,844.26,1093.71,704785.63,721,89,0,40,118,...,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,63,299,0x0020731604c882cf7bf8c444be97d17b19ea4316
3,2,0,12709.07,2958.44,1218216.73,94,8,0,5,14,...,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,154,257,0x0002b44ddb1476db43c868bd494422ee4c136fed
4,2,0,12709.07,2958.44,1218216.73,94,8,0,5,14,...,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,154,257,0x4c1da8781f6ca312bc11217b3f61e5dfdf428de1


In [17]:
# --- 2.1.2. Tải mô hình và tạo giải thích xAI mẫu (nếu chưa có API) ---
# Tạm thời, chúng ta sẽ tải mô hình và tạo một giải thích xAI mẫu cho một địa chỉ để đưa vào KB.
# Trong triển khai thực tế, bạn sẽ có một API riêng cho Anomaly Detection Module.
try:
    model_anomaly = joblib.load(TRAINED_MODEL_PATH)
    print("Đã tải mô hình phát hiện bất thường.")
except FileNotFoundError:
    print(f"Cảnh báo: Không tìm thấy mô hình tại {TRAINED_MODEL_PATH}. Sẽ không tạo giải thích xAI mẫu.")
    model_anomaly = None

# Hàm tạo giải thích xAI từ output SHAP (ví dụ)
def generate_xai_explanation(address_data, shap_values_output):
    if not shap_values_output:
        return "Không có giải thích chi tiết cho địa chỉ này."

    explanation_text = f"Giải thích dự đoán cho địa chỉ {address_data['Address']}: "
    explanation_text += "Các yếu tố quan trọng ảnh hưởng đến dự đoán (từ mô hình phát hiện bất thường) bao gồm:\n"
    
    # Sắp xếp theo độ quan trọng giảm dần
    sorted_shap = sorted(shap_values_output, key=lambda x: abs(x[0]), reverse=True)
    
    for importance, feature_name in sorted_shap:
        # Lấy giá trị thực của feature nếu có trong address_data
        feature_value = address_data.get(feature_name, "không xác định")
        explanation_text += f"- **{feature_name}**: Giá trị ảnh hưởng {importance:.4f}. (Giá trị thực: {feature_value})\n"
    
    return explanation_text


Đã tải mô hình phát hiện bất thường.


d:\Anacoda\envs\lightning_env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Anacoda\envs\lightning_env\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
source_docs = []

for index, row in df_data.iterrows():
    address = row['Address']
    flag = int(row['FLAG']) # Chắc chắn là int
    
    # Tạo mô tả chung về địa chỉ từ các đặc trưng
    description_text = f"Thông tin chi tiết về địa chỉ ví Ethereum: {address}. " \
                       f"Địa chỉ này được phân loại là {'bất thường' if flag == 1 else 'hợp lệ'} (FLAG {flag}). " \
                       f"Thời gian hoạt động từ giao dịch đầu tiên đến cuối cùng là {row['Time Diff between first and last (Mins)']:.2f} phút. " \
                       f"Đã gửi {int(row['Sent tnx'])} giao dịch và nhận {int(row['Received Tnx'])} giao dịch. " \
                       f"Tổng Ether đã gửi: {row['total Ether sent']:.6f} ETH, tổng Ether đã nhận: {row['total ether received']:.6f} ETH. " \
                       f"Số dư Ether cuối cùng: {row['total ether balance']:.6f} ETH. " \
                       f"Tổng số giao dịch ERC20: {int(row['Total ERC20 tnxs'])}. " \
                       f"Loại token ERC20 gửi nhiều nhất: {row['ERC20 most sent token type_label']} và nhận nhiều nhất: {row['ERC20_most_rec_token_type_label']}. " \
                       f"Giá trị ERC20 trung bình nhận được: {row['ERC20 avg val rec']:.6f}."
    
    # Thêm các đặc trưng đồ thị nếu bạn đã tính toán và thêm vào df_data
    # Ví dụ: if 'pagerank' in row: description_text += f" Điểm PageRank: {row['pagerank']:.4f}."